In [ ]:
import pandas as pd
import astropy.units as u
import warnings
import time
from astropy.coordinates import SkyCoord
from astroquery.sdss import SDSS
from astroquery.ned import Ned

# def text(index_begin=0):
data_original = pd.read_csv('irsa_catalog_search_results.tbl', sep='\s+', skiprows=32, header=None,
                            names=['name', 'band', 'ra', 'dec', 'raerr', 'decerr', 'flux', 'type'])
# data_original=pd.read_csv('text.tbl',sep='\s+',header=None,names=['name','band','ra','dec','raerr','decerr','flux','type'])
index = len(data_original.index.values)


# <MaskedColumn name='Type' dtype='object' description="NED's
# Preferred Object Type: G, GPair, GTrpl, GGroup, GClstr, QSO, AbLS, RadioS, IrS, EmLS, UvES, XrayS, SN"

def detect(index_begin=0, interval=1000):

    for i in range(index_begin, index):
        print('The echo is', i, '/', index)
        ra = data_original['ra'][i]  # 赤经
        dec = data_original['dec'][i]  # 赤纬
        try:
            table_sdss = SDSS.query_crossid(SkyCoord(ra, dec, unit=(u.deg, u.deg)), radius=3 * u.arcsec,
                                            data_release=15)
            warnings.filterwarnings('ignore')
            if table_sdss is None:
                table_ned = Ned.query_region(SkyCoord(ra, dec, unit=(u.deg, u.deg)), radius=3 * u.arcsec,
                                             equinox='J2000.0')
                if len(table_ned) > 0:
                    # data_original['type'][i] = table_ned['Type'].description
                    data_original['type'][i] = table_ned['Type'][0]
                else:
                    continue
            else:
                data_original['type'][i] = table_sdss['type'][0]
        except:
            continue

        if (i % interval) == 0 and i != 0 :   # 默认每查询500次保存一次文件
            data_name = 'result_%s.csv' % (i)
            data_original.to_csv(data_name, float_format="%.6f", index=0)
            print('The file %s has been saved.\n' % data_name)
            time.sleep(5)   # 提示信息延时5秒
